In [16]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable('json')
alt.renderers.enable('jupyterlab')

RendererRegistry.enable('jupyterlab')

# Load Data

In [17]:
start_date = '20221130'
end_date = '20230509'

tidy_dataset_fn = f"tidy_df_{start_date}_{end_date}_noplanar_fit.parquet"
tidy_daily_dataset_output_fn = f"tidy_df_daily_{start_date}_{end_date}_noplanar_fit.parquet"

In [18]:
try:
    tidy_df_5Min = pd.read_parquet(
        tidy_dataset_fn
    )
except FileNotFoundError:
    print("No file such file exists for these dates.")
tidy_df_5Min['time'] = pd.to_datetime(tidy_df_5Min['time'])

In [19]:
z0_andreas_df = pd.read_parquet("z0estimates/z0_andreas_df.parquet")
z0_andreas_df_strict = pd.read_parquet("z0estimates/z0_andreas_df_strict.parquet")
z0_andreas_df_weekly = pd.read_parquet("z0estimates/z0_andreas_df_weekly.parquet")

In [20]:
coare_model_results_entire_df = pd.read_parquet("coare_model_results.parquet")
coare_model_results_df = coare_model_results_entire_df[
    coare_model_results_entire_df.config == 'Tsurf_c e_sat_alduchov z0_andreas_weekly'
]

# Analyze results

In [21]:
src = tidy_df_5Min.query("variable == 'L_3m_c'").set_index('time').rename(
    columns = {'value': 'L_3m_c'}
)[['L_3m_c']].join(
    tidy_df_5Min.query("variable == 'u*_3m_c'").set_index('time').rename(
        columns = {'value': 'ustar_3m_c'}
    )[['ustar_3m_c']]
).join(
    z0_andreas_df.set_index('time')
)
# positive - stable
# negative - unstable
src['stability'] = src['L_3m_c'].apply(lambda L: 'stable' if L > 0 else 'unstable')
src.head()

,L_3m_c,ustar_3m_c,z0,z0T,z0q,stability
time,,,,,,
2022-11-29 17:02:30,-16.815512,0.377098,0.076182,2050.978978,1.972075,unstable
2022-11-29 17:07:30,-4.879467,0.249449,0.019717,94.869536,1.326326,unstable
2022-11-29 17:12:30,-2.556141,0.265064,0.006696,12.654200,0.797514,unstable
2022-11-29 17:17:30,-2.655310,0.301103,0.005653,2.416952,0.940579,unstable
2022-11-29 17:22:30,-2.447218,0.375441,0.015371,0.969892,0.833297,unstable


In [35]:
model_src = src.groupby(pd.Grouper(freq='30Min')).mean(numeric_only=True).join(
    coare_model_results_df[['usr']]
)
model_src['stability'] = model_src['L_3m_c'].apply(lambda L: 'stable' if L > 0 else 'unstable')

# Compare measured z0 with measured $u_*$

In [6]:
(
# Chart 1
alt.Chart(
    src.query("ustar_3m_c < 0.7").query("z0 > 0.00000001")
).mark_circle(
    size=3,
    opacity=0.25
).encode(
    alt.X("ustar_3m_c").title("u* at 3 meters on tower c"),
    alt.Y("z0").title("Measured z_0 (m)").scale(type='log')
) +
# Chart 2
alt.Chart(
    src.query("ustar_3m_c < 0.7").query("z0 > 0.00000001")
).mark_boxplot(
    opacity=0.5,
    color='black', 
    extent='min-max'
).encode(
    alt.X("ustar_3m_c").title("u* at 3 meters on tower c").bin(maxbins=35),
    alt.Y("z0").title("Measured z_0 (m)").scale(type='log')
)
).configure_axis(grid=False)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [7]:
alt.layer(
    alt.Chart().mark_circle(
        size=3,
        opacity=0.25
    ).encode(
        alt.X("ustar_3m_c").title("u* at 3 meters on tower c"),
        alt.Y("z0").title("Measured z_0 (m)").scale(type='log')
    ) +
    # Chart 2
    alt.Chart().mark_boxplot(
        opacity=0.5,
        color='black', 
        extent='min-max'
    ).encode(
        alt.X("ustar_3m_c").title("u* at 3 meters on tower c").bin(maxbins=35),
        alt.Y("z0").title("Measured z_0 (m)").scale(type='log')
    ),
    data = src.query("ustar_3m_c < 0.7").query("z0 > 0.00000001")
).facet("stability")

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


# Compare measured z0 with modeled $u_*$

In [38]:
(
# Chart 1
alt.Chart(
    model_src.query("usr < 0.7").query("z0 > 0.00000001")
).mark_circle(
    size=3,
    opacity=0.25
).encode(
    alt.X("usr").title("u* at 3 meters on tower c"),
    alt.Y("z0").title("Measured z_0 (m)").scale(type='log')
) +
# Chart 2
alt.Chart(
    model_src.query("usr < 0.7").query("z0 > 0.00000001")
).mark_boxplot(
    opacity=0.5,
    color='black', 
    extent='min-max'
).encode(
    alt.X("usr").title("u* at 3 meters on tower c").bin(maxbins=35),
    alt.Y("z0").title("Measured z_0 (m)").scale(type='log')
)
).configure_axis(grid=False)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [37]:
alt.layer(
    alt.Chart().mark_circle(
        size=3,
        opacity=0.25
    ).encode(
        alt.X("usr").title("u* at 3 meters on tower c"),
        alt.Y("z0").title("Measured z_0 (m)").scale(type='log')
    ) +
    # Chart 2
    alt.Chart().mark_boxplot(
        opacity=0.5,
        color='black', 
        extent='min-max'
    ).encode(
        alt.X("usr").title("u* at 3 meters on tower c").bin(maxbins=35),
        alt.Y("z0").title("Measured z_0 (m)").scale(type='log')
    ),
    data = model_src.query("usr < 0.7").query("z0 > 0.00000001")
).facet("stability")

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [8]:
(
# Chart 1
alt.Chart(
    src.query("stability == 'unstable'").query("ustar_3m_c < 0.7").query("z0 > 0.00000001")
).mark_circle(
    size=3,
    opacity=0.25
).encode(
    alt.X("ustar_3m_c").title("u* at 3 meters on tower c"),
    alt.Y("z0").title("Measured z_0 (m)").scale(type='log')
) +
# Chart 2
alt.Chart(
    src.query("stability == 'unstable'").query("ustar_3m_c < 0.7").query("z0 > 0.00000001")
).mark_boxplot(
    opacity=0.5,
    color='black', 
    extent='min-max'
).encode(
    alt.X("ustar_3m_c").title("u* at 3 meters on tower c").bin(maxbins=35),
    alt.Y("z0").title("Measured z_0 (m)").scale(type='log')
)
).configure_axis(grid=False)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [9]:
src['ustar_3m_c bin'] = pd.cut(src['ustar_3m_c'], np.arange(0.0, 0.7, 0.02))
src['ustar_3m_c bin'] = src['ustar_3m_c bin'].apply(lambda x: x.left+0.01)
import scipy.stats as stats
gmeans = pd.DataFrame(src.groupby('ustar_3m_c bin')['z0'].apply(lambda x: stats.gmean(x))).reset_index()

gmeans_stable = pd.DataFrame(src.query("L_3m_c > 0").groupby('ustar_3m_c bin')['z0'].apply(lambda x: stats.gmean(x))).reset_index()
gmeans_unstable = pd.DataFrame(src.query("L_3m_c < 0").groupby('ustar_3m_c bin')['z0'].apply(lambda x: stats.gmean(x))).reset_index()

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_28530/620765223.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  gmeans = pd.DataFrame(src.groupby('ustar_3m_c bin')['z0'].apply(lambda x: stats.gmean(x))).reset_index()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_28530/620765223.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  gmeans_stable = pd.DataFrame(src.query("L_3m_c > 0").groupby('ustar_3m_c bin')['z0'].apply(lambda x: stats.gmean(x))).reset_index()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_28530/620765223.py:7: FutureWarning: The default of observed=False

In [27]:
src

,L_3m_c,ustar_3m_c,z0,z0T,z0q,stability
time,,,,,,
2022-11-29 17:02:30,-16.815512,0.377098,0.076182,2.050979e+03,1.972075,unstable
2022-11-29 17:07:30,-4.879467,0.249449,0.019717,9.486954e+01,1.326326,unstable
2022-11-29 17:12:30,-2.556141,0.265064,0.006696,1.265420e+01,0.797514,unstable
2022-11-29 17:17:30,-2.655310,0.301103,0.005653,2.416952e+00,0.940579,unstable
2022-11-29 17:22:30,-2.447218,0.375441,0.015371,9.698918e-01,0.833297,unstable
...,...,...,...,...,...,...
2023-05-09 17:37:30,0.142787,0.143017,NaN,4.785412e-15,0.000002,stable
2023-05-09 17:42:30,-2.488529,0.262250,0.010463,3.976248e+45,1.146755,unstable
2023-05-09 17:47:30,0.167794,0.186958,NaN,2.574417e-12,0.000007,stable


In [24]:
(
    (
        alt.Chart(
            src.query("L_3m_c > 0").query("ustar_3m_c < 0.7").query("z0 > 0.00000001")
        ).mark_circle(
            size=3,
            opacity=0.25
        ).encode(
            alt.X("ustar_3m_c").title("u* at 3 meters on tower c"),
            alt.Y("z0").title("Measured z_0 (m)").scale(type='log')
        ) +\
        alt.Chart(gmeans_stable).mark_circle().encode(
            alt.X("ustar_3m_c bin:Q"),
            alt.Y("z0").scale(type='log')
        )
    ).properties(title='Stable') | (
        alt.Chart(
            src.query("L_3m_c < 0").query("ustar_3m_c < 0.7").query("z0 > 0.00000001")
        ).mark_circle(
            size=3,
            opacity=0.25
        ).encode(
            alt.X("ustar_3m_c").title("u* at 3 meters on tower c"),
            alt.Y("z0").title("Measured z_0 (m)").scale(type='log')
        ) +\
        alt.Chart(gmeans_unstable).mark_circle().encode(
            alt.X("ustar_3m_c bin:Q"),
            alt.Y("z0").scale(type='log')
        )
    ).properties(title="Unstable")

).resolve_scale(y='shared', x='shared')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting
